In [144]:
import binascii
import numpy
import os
import errno
from PIL import Image
from array import *
from random import shuffle

PNG_SIZE=28
examplepath=r"C:\Users\b-byr\Desktop\Stage\Project\PCAPs\USTC-TK2016\3_ProcessedSession\TrimedSession\Test\packets_20200612_000001-L7\packets_20200612_000001.pcap.TCP_3-233-116-190_80_192-168-20-29_44524.bin"

In [96]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [121]:
def GetMatrixFrom(filename):
    with open(filename, 'rb') as f:
        content = f.read()
        hexst = binascii.hexlify(content)
        fh = numpy.array([int(hexst[i:i+2],16) for i in range(0, len(hexst), 2)])
        rn = len(fh)//28
        fh = numpy.reshape(fh[:rn*28],(-1,28))
        fh = numpy.uint8(fh)
    return fh

In [141]:
paths = [['3_ProcessedSession\TrimedSession\Train', '4_Png\Train'],['3_ProcessedSession\TrimedSession\Test', '4_Png\Test']]
for p in paths:
    for i, d in enumerate(os.listdir(p[0])):
        dir_full = os.path.join(p[1], str(i))
        print(dir_full)
        mkdir_p(dir_full)
        for f in os.listdir(os.path.join(p[0], d)):
            bin_full = os.path.join(p[0], d, f)
            im = Image.fromarray(GetMatrixFrom(bin_full))
            png_full=os.path.join(dir_full, os.path.splitext(f)[0]+'.png')
            im.save(png_full)


4_Png\Train\0
4_Png\Test\0


In [136]:
 im = Image.fromarray(GetMatrixFrom(examplepath))

In [128]:
im

In [145]:
# Load from and save to
mkdir_p('5_Mnist')
Names = [['4_Png\Train','5_Mnist\\train']]

for name in Names:	
    data_image = array('B')
    data_label = array('B')

    FileList = []
    for dirname in os.listdir(name[0]): 
        path = os.path.join(name[0],dirname)
        for filename in os.listdir(path):
            if filename.endswith(".png"):
                FileList.append(os.path.join(name[0],dirname,filename))

    shuffle(FileList) # Usefull for further segmenting the validation set

    for filename in FileList:
        print(filename)
        label = int(filename.split('\\')[2])
        Im = Image.open(filename)
        pixel = Im.load()
        width, height = Im.size
        for x in range(0,width):
            for y in range(0,height):
                data_image.append(pixel[y,x])
        data_label.append(label) # labels start (one unsigned byte each)
    hexval = "{0:#0{1}x}".format(len(FileList),6) # number of files in HEX
    hexval = '0x' + hexval[2:].zfill(8)
    
    # header for label array
    header = array('B')
    header.extend([0,0,8,1])
    header.append(int('0x'+hexval[2:][0:2],16))
    header.append(int('0x'+hexval[2:][2:4],16))
    header.append(int('0x'+hexval[2:][4:6],16))
    header.append(int('0x'+hexval[2:][6:8],16))	
    data_label = header + data_label

    # additional header for images array
    if max([width,height]) <= 256:
        header.extend([0,0,0,width,0,0,0,height])
    else:
        raise ValueError('Image exceeds maximum size: 256x256 pixels');

    header[3] = 3 # Changing MSB for image data (0x00000803)
    data_image = header + data_image
    output_file = open(name[1]+'-images-idx3-ubyte', 'wb')
    data_image.tofile(output_file)
    output_file.close()
    output_file = open(name[1]+'-labels-idx1-ubyte', 'wb')
    data_label.tofile(output_file)
    output_file.close()

# gzip resulting files
for name in Names:
    os.system('gzip '+name[1]+'-images-idx3-ubyte')
    os.system('gzip '+name[1]+'-labels-idx1-ubyte')

4_Png\Train\0\packets_20200612_000001.pcap.TCP_192-168-20-15_48574_193-51-224-8_80.png
4_Png\Train\0\packets_20200612_000001.pcap.TCP_192-168-20-15_42172_216-58-204-138_443.png
4_Png\Train\0\packets_20200612_000001.pcap.UDP_8-8-8-8_53_192-168-20-18_54262.png
4_Png\Train\0\packets_20200612_000001.pcap.UDP_8-8-8-8_53_192-168-20-18_34071.png
4_Png\Train\0\packets_20200612_000001.pcap.TCP_34-193-17-185_80_192-168-20-29_55284.png
4_Png\Train\0\packets_20200612_000001.pcap.UDP_8-8-8-8_53_192-168-20-29_15492.png
4_Png\Train\0\packets_20200612_000001.pcap.TCP_52-21-35-120_80_192-168-20-29_57140.png
4_Png\Train\0\packets_20200612_000001.pcap.TCP_172-217-19-238_443_192-168-20-18_40755.png
4_Png\Train\0\packets_20200612_000001.pcap.TCP_13-225-34-164_443_192-168-20-29_59476.png
4_Png\Train\0\packets_20200612_000001.pcap.TCP_192-168-20-15_43092_216-58-201-229_443.png
4_Png\Train\0\packets_20200612_000001.pcap.UDP_8-8-8-8_53_192-168-20-15_41459.png
4_Png\Train\0\packets_20200612_000001.pcap.UDP_8-8-